# ⏰ Time Series Forecasting with LSTM

Welcome to **Time Series Forecasting**! In this notebook, we'll use LSTM networks to predict future values from sequential data. We'll work with stock prices, weather data, and energy consumption.

## What you'll learn:
- Time series data preprocessing
- Sliding window technique
- LSTM architectures for forecasting
- Multi-step and multi-variate predictions

Let's predict the future! 🔮

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import yfinance as yf

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

plt.style.use('seaborn-v0_8')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

In [ ]:
# Load stock price data
def load_stock_data(symbol='AAPL', period='2y'):
    """Load stock data from Yahoo Finance"""
    try:
        stock = yf.download(symbol, period=period)
        return stock
    except:
        # Fallback: Generate synthetic stock data
        print("Using synthetic stock data...")
        dates = pd.date_range(start='2022-01-01', end='2024-01-01', freq='D')
        np.random.seed(42)
        price = 150 + np.cumsum(np.random.randn(len(dates)) * 0.5)
        volume = np.random.randint(1000000, 10000000, len(dates))
        
        data = pd.DataFrame({
            'Open': price + np.random.randn(len(dates)) * 2,
            'High': price + np.abs(np.random.randn(len(dates)) * 3),
            'Low': price - np.abs(np.random.randn(len(dates)) * 3),
            'Close': price,
            'Volume': volume
        }, index=dates)
        return data

# Load data
stock_data = load_stock_data('AAPL', '2y')
print(f"Stock data shape: {stock_data.shape}")
print(f"Date range: {stock_data.index[0]} to {stock_data.index[-1]}")

# Display first few rows
print("\n📊 Stock Data Sample:")
print(stock_data.head())

In [ ]:
# Visualize stock data
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Price chart
axes[0, 0].plot(stock_data.index, stock_data['Close'], label='Close Price')
axes[0, 0].plot(stock_data.index, stock_data['Open'], alpha=0.7, label='Open Price')
axes[0, 0].set_title('📈 Stock Price Over Time')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Price ($)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Volume chart
axes[0, 1].plot(stock_data.index, stock_data['Volume'], color='orange')
axes[0, 1].set_title('📊 Trading Volume')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Volume')
axes[0, 1].grid(True, alpha=0.3)

# Daily returns
returns = stock_data['Close'].pct_change().dropna()
axes[1, 0].plot(returns.index, returns, alpha=0.7)
axes[1, 0].set_title('📉 Daily Returns')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Return (%)')
axes[1, 0].grid(True, alpha=0.3)

# Returns distribution
axes[1, 1].hist(returns, bins=50, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('📊 Returns Distribution')
axes[1, 1].set_xlabel('Daily Return')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Statistics:")
print(f"Average daily return: {returns.mean():.4f}")
print(f"Volatility (std): {returns.std():.4f}")
print(f"Min/Max prices: ${stock_data['Close'].min():.2f} / ${stock_data['Close'].max():.2f}")

In [ ]:
# Prepare data for LSTM
def create_sequences(data, seq_length, target_col='Close'):
    """Create sequences for LSTM training"""
    sequences = []
    targets = []
    
    for i in range(seq_length, len(data)):
        sequences.append(data[i-seq_length:i])
        targets.append(data[i][target_col] if isinstance(data, pd.DataFrame) else data[i])
    
    return np.array(sequences), np.array(targets)

# Use only Close price for univariate forecasting
close_prices = stock_data['Close'].values.reshape(-1, 1)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(close_prices)

# Create sequences
seq_length = 60  # Use 60 days to predict next day
X, y = create_sequences(scaled_data.flatten(), seq_length)

# Reshape X for LSTM (samples, time steps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# Split data (80% train, 20% test)
split_idx = int(0.8 * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Sequence length: {seq_length} days")
print(f"Total sequences: {len(X)}")

In [ ]:
# Model 1: Vanilla LSTM
def create_vanilla_lstm(seq_length, n_features=1):
    model = models.Sequential([
        layers.LSTM(50, input_shape=(seq_length, n_features)),
        layers.Dropout(0.2),
        layers.Dense(1)
    ])
    return model

vanilla_lstm = create_vanilla_lstm(seq_length)
vanilla_lstm.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("🔄 Vanilla LSTM Architecture:")
vanilla_lstm.summary()

In [ ]:
# Train Vanilla LSTM
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-7)
]

print("🚀 Training Vanilla LSTM...")
history_vanilla = vanilla_lstm.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

# Make predictions
train_pred = vanilla_lstm.predict(X_train)
test_pred = vanilla_lstm.predict(X_test)

# Inverse transform predictions
train_pred = scaler.inverse_transform(train_pred)
test_pred = scaler.inverse_transform(test_pred)
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate metrics
train_mae = mean_absolute_error(y_train_actual, train_pred)
test_mae = mean_absolute_error(y_test_actual, test_pred)
train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_pred))

print(f"\n🎯 Vanilla LSTM Results:")
print(f"Train MAE: ${train_mae:.2f}, RMSE: ${train_rmse:.2f}")
print(f"Test MAE: ${test_mae:.2f}, RMSE: ${test_rmse:.2f}")

In [ ]:
# Model 2: Stacked LSTM
def create_stacked_lstm(seq_length, n_features=1):
    model = models.Sequential([
        layers.LSTM(50, return_sequences=True, input_shape=(seq_length, n_features)),
        layers.Dropout(0.2),
        layers.LSTM(50, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(50),
        layers.Dropout(0.2),
        layers.Dense(1)
    ])
    return model

stacked_lstm = create_stacked_lstm(seq_length)
stacked_lstm.compile(optimizer='adam', loss='mse', metrics=['mae'])

print("🏗️ Stacked LSTM Architecture:")
stacked_lstm.summary()

In [ ]:
# Train Stacked LSTM
print("🚀 Training Stacked LSTM...")
history_stacked = stacked_lstm.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

# Make predictions
stacked_test_pred = stacked_lstm.predict(X_test)
stacked_test_pred = scaler.inverse_transform(stacked_test_pred)

# Calculate metrics
stacked_test_mae = mean_absolute_error(y_test_actual, stacked_test_pred)
stacked_test_rmse = np.sqrt(mean_squared_error(y_test_actual, stacked_test_pred))

print(f"\n🎯 Stacked LSTM Results:")
print(f"Test MAE: ${stacked_test_mae:.2f}, RMSE: ${stacked_test_rmse:.2f}")

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Training history - Vanilla LSTM
axes[0, 0].plot(history_vanilla.history['loss'], label='Training Loss')
axes[0, 0].plot(history_vanilla.history['val_loss'], label='Validation Loss')
axes[0, 0].set_title('📉 Vanilla LSTM - Training History')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Training history - Stacked LSTM
axes[0, 1].plot(history_stacked.history['loss'], label='Training Loss')
axes[0, 1].plot(history_stacked.history['val_loss'], label='Validation Loss')
axes[0, 1].set_title('📉 Stacked LSTM - Training History')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Predictions comparison
test_dates = stock_data.index[split_idx + seq_length:]
axes[1, 0].plot(test_dates, y_test_actual, label='Actual', alpha=0.8)
axes[1, 0].plot(test_dates, test_pred, label='Vanilla LSTM', alpha=0.8)
axes[1, 0].plot(test_dates, stacked_test_pred, label='Stacked LSTM', alpha=0.8)
axes[1, 0].set_title('📈 Stock Price Predictions')
axes[1, 0].set_xlabel('Date')
axes[1, 0].set_ylabel('Price ($)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Error analysis
vanilla_errors = y_test_actual.flatten() - test_pred.flatten()
stacked_errors = y_test_actual.flatten() - stacked_test_pred.flatten()

axes[1, 1].hist(vanilla_errors, bins=30, alpha=0.7, label='Vanilla LSTM', edgecolor='black')
axes[1, 1].hist(stacked_errors, bins=30, alpha=0.7, label='Stacked LSTM', edgecolor='black')
axes[1, 1].set_title('📊 Prediction Errors Distribution')
axes[1, 1].set_xlabel('Error ($)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n🏆 Model Comparison:")
print(f"Vanilla LSTM - MAE: ${test_mae:.2f}, RMSE: ${test_rmse:.2f}")
print(f"Stacked LSTM - MAE: ${stacked_test_mae:.2f}, RMSE: ${stacked_test_rmse:.2f}")
improvement = ((test_mae - stacked_test_mae) / test_mae) * 100
print(f"Improvement: {improvement:.1f}%")

## 🎉 Congratulations!

You've mastered time series forecasting with LSTMs! Here's what you've accomplished:

✅ **Time Series Processing**: Data preparation and windowing  
✅ **LSTM Architectures**: Vanilla and stacked models  
✅ **Forecasting**: Single-step price prediction  
✅ **Model Evaluation**: MAE, RMSE, and error analysis  

### 🚀 Next Steps:
1. Try multi-variate forecasting with multiple features
2. Implement multi-step ahead predictions
3. Experiment with attention mechanisms
4. Move on to **Project 06: GAN for Face Generation**

Ready for generative models? Let's create new data! 🎨